# Project 6: Determination of the Ground-State of the Lennard Jones Potential
This project uses a Monte-Carlo Variational method to find the approximate wavefunction of an arbitrary potential in 1-D.

## Objective:

The interaction between neutral, nonpolarized atoms is modelled by the so-called _Lennard-Jones Potential_, which is a radial potential given by

$$V(r) = 4\varepsilon \left(\left[\frac{\sigma}{r}\right]^{12}-\left[\frac{\sigma}{r}\right]^{6}\right)$$

where $\varepsilon$ is the bonding energy, $\sigma$ is the van der Waals radius, and $r$ is the separation distance between the two particles.

1. 